**Install & import required packages**

In [4]:
!pip install transformers
!pip install transformers torch
!pip install datasets
!pip install evaluate

import requests
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from PIL import Image
from transformers import AutoProcessor, AutoModelForSeq2SeqLM, AutoModelForVision2Seq, AutoTokenizer, AutoModelForCausalLM

**Load models**

In [5]:
# img2word model (kosmos) from https://huggingface.co/microsoft/led-base-16384
kosmos_name = "microsoft/kosmos-2-patch14-224"
kosmos_model = AutoModelForVision2Seq.from_pretrained(kosmos_name)
kosmos_processor = AutoProcessor.from_pretrained(kosmos_name)

# word2haiku model (gpt2) from https://huggingface.co/fabianmmueller/deep-haiku-gpt-2
# gpt2_name = "microsoft/kosmos-2-patch14-224"
# gpt2_tokenizer = AutoTokenizer.from_pretrained(gpt2_name)
# gpt2_model = AutoModelForCausalLM.from_pretrained(gpt2_name)
# giving error ValueError: Unrecognized configuration class <class 'transformers.models.kosmos2.configuration_kosmos2.Kosmos2Config'> for this kind of AutoModel: AutoModelForCausalLM.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


ValueError: Unrecognized configuration class <class 'transformers.models.kosmos2.configuration_kosmos2.Kosmos2Config'> for this kind of AutoModel: AutoModelForCausalLM.
Model type should be one of BartConfig, BertConfig, BertGenerationConfig, BigBirdConfig, BigBirdPegasusConfig, BioGptConfig, BlenderbotConfig, BlenderbotSmallConfig, BloomConfig, CamembertConfig, LlamaConfig, CodeGenConfig, CpmAntConfig, CTRLConfig, Data2VecTextConfig, ElectraConfig, ErnieConfig, FalconConfig, FuyuConfig, GitConfig, GPT2Config, GPT2Config, GPTBigCodeConfig, GPTNeoConfig, GPTNeoXConfig, GPTNeoXJapaneseConfig, GPTJConfig, LlamaConfig, MarianConfig, MBartConfig, MegaConfig, MegatronBertConfig, MistralConfig, MptConfig, MusicgenConfig, MvpConfig, OpenLlamaConfig, OpenAIGPTConfig, OPTConfig, PegasusConfig, PersimmonConfig, PLBartConfig, ProphetNetConfig, QDQBertConfig, ReformerConfig, RemBertConfig, RobertaConfig, RobertaPreLayerNormConfig, RoCBertConfig, RoFormerConfig, RwkvConfig, Speech2Text2Config, Trans

**Image2Word**

In [6]:
prompt = "<grounding>An image of"

# User inputted image
image = Image.open("data/pig.jpeg")

inputs = kosmos_processor(text=prompt, images=image, return_tensors="pt")

generated_ids = kosmos_model.generate(
    pixel_values=inputs["pixel_values"],
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    image_embeds=None,
    image_embeds_position_mask=inputs["image_embeds_position_mask"],
    use_cache=True,
    max_new_tokens=128,
)
generated_text = kosmos_processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

# Specify `cleanup_and_extract=False` in order to see the raw model generation.
processed_text = kosmos_processor.post_process_generation(generated_text, cleanup_and_extract=False)

# print(processed_text)
# `<grounding> An image of<phrase> a snowman</phrase><object><patch_index_0044><patch_index_0863></object> warming himself by<phrase> a fire</phrase><object><patch_index_0005><patch_index_0911></object>.`

# By default, the generated  text is cleanup and the entities are extracted.
processed_text, entities = kosmos_processor.post_process_generation(generated_text)

# print(processed_text)
# `An image of a snowman warming himself by a fire.`

# print(entities)
# `[('a snowman', (12, 21), [(0.390625, 0.046875, 0.984375, 0.828125)]), ('a fire', (41, 47), [(0.171875, 0.015625, 0.484375, 0.890625)])]`

kosmos_ouput = entities[0][0]
print(kosmos_ouput)

a piglet in a field


**Pass kosmos output to word2haiku model**

In [ ]:
#Wworrrrrdddd 22222222 hiakauauakuuuuu